In [1]:
# Roberta attempt
# Anita Sun

In [22]:
import sqlite3
import pandas as pd

In [14]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
import torch
import tensorflow.keras as tf_keras

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='emotion'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
'''

model = AutoModelForSequenceClassification.from_pretrained(MODEL, trust_remote_code=True)
model.save_pretrained(MODEL)

text = "Celebrating my promotion 😎"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)


'''

# # TF
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

text = "Celebrating my promotion 😎"
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)
scores = output[0][0].numpy()
scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

tf_model.h5:   0%|          | 0.00/501M [00:00<?, ?B/s]

2025-02-27 12:45:21.449006: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-emotion.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


1) joy 0.9382
2) optimism 0.0362
3) anger 0.0145
4) sadness 0.0112


In [23]:
conn = sqlite3.connect('raam_database.db', timeout=10)

In [24]:
cur = conn.cursor()

In [25]:
q = '''
    SELECT name FROM sqlite_master WHERE type="table"
    '''

cur.execute(q).fetchall()

[('health_anxiety_table_raw',),
 ('health_anxiety_posts',),
 ('sqlite_sequence',),
 ('health_anxiety_comments',),
 ('nursing_table_raw',),
 ('nursing_posts',),
 ('nursing_comments',),
 ('teaching_table_raw',),
 ('teaching_posts',),
 ('teaching_comments',)]

In [26]:
q2 = '''
    PRAGMA table_info(teaching_posts)
    '''

cur.execute(q2).fetchall()

[(0, 'post_id', 'INTEGER', 0, None, 1),
 (1, 'timestamp', 'TEXT', 0, None, 0),
 (2, 'username', 'TEXT', 0, None, 0),
 (3, 'post_title', 'TEXT', 0, None, 0),
 (4, 'post_body', 'TEXT', 0, None, 0),
 (5, 'score', 'TEXT', 0, None, 0),
 (6, 'comments_count', 'TEXT', 0, None, 0),
 (7, 'post_url', 'TEXT', 0, None, 0)]

In [44]:
q3 = '''
    SELECT post_body, post_id FROM teaching_posts
    
    '''

teaching_post_lst = cur.execute(q3).fetchall()

In [40]:
labels

['anger', 'joy', 'optimism', 'sadness']

In [71]:
df = pd.DataFrame({'post_id': [], 'body_text': [],
                   'anger': [], 'joy': [],
                   'optimism': [], 'sadness': []})

In [65]:
for i in range(27):
    print(teaching_post_lst[i])

('Please let the door hit you on your big dumb head on the way out!', 1)
("Sorry but your kids are crazy because that's how they were raised, and we get to deal with that every day, thanks!", 2)
('Not a drill. I have a tool kit in my classroom. I passed out hammers and wrenches and screw drivers. I was a drill sergeant. YOU flip the table up against the door. YOU here pass out scissors. YOU grab that chair. \nThe kids are freshmen in high school. They were ready to fight. We were ready to fight for an hour and a half while we waited in the dark. I cussed a lot. We had a couple giggles. I was absolutely terrified I would have to make the choice between my own life and a student’s today. \nAfter an hour and a half, we found out it was a false threat. No gun in the building. Afterwards kids hugged me and told me what I meant to them. I tried not to cry. \nThey didn’t let the kids go home after unless their parents were there. They were expected to go to class. I only had prep left. I have

In [64]:
print(teaching_post_lst[27])

('I went into this school year ready for anything. That "anything" didn\'t include 95% of what hit me like a freight train in just my first week. The kids that got dropped in my lap this year were beyond anything I ever expected. \nOn day 1, I had a student (5th Grade ELA) stand on his desk and beat on his chest while screaming absolute nonsense. It wasn\'t even words. It was just guttural noise. This student proceeded to jump down and crawl on the floor across my classroom. All I could do was stand there and think "What the absolute fuck is happening?". This same phrase has now become a constant in my mind.\nI spent my first entire month crying after work on my drive home. Between the kids, constant crazy expectations, PD days everytime I turned around, and the "can you help out with this or that", I was dying. I seriously went to my doctor and asked him to please give whatever drugs, even experimental, you can to just help me handle all of this. *He didn\'t do that, but did adjust so

In [72]:
for weird_tup in teaching_post_lst:
    string = weird_tup[0]
    string = preprocess(string)
    id = weird_tup[1]
    tokenized = tokenizer(string, return_tensors='tf', truncation=True, max_length=512)
    output = model(tokenized)
    scores = output[0][0].numpy()
    scores = softmax(scores)
    print(scores, labels)
    df.loc[len(df)] = [id, string, np.nan, np.nan, np.nan, np.nan]

    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    print(string)

    df.loc[len(df)-1, labels] = scores

    for i in range(scores.shape[0]):
        l = labels[ranking[i]]
        s = scores[ranking[i]]
        #df.iloc[-1, l] = s
        print(f"{i+1}) {l} {np.round(float(s), 4)}")

[0.9187739  0.01099181 0.03515448 0.03507975] ['anger', 'joy', 'optimism', 'sadness']
Please let the door hit you on your big dumb head on the way out!
1) anger 0.9188
2) optimism 0.0352
3) sadness 0.0351
4) joy 0.011
[0.5995315  0.05724264 0.21298192 0.13024396] ['anger', 'joy', 'optimism', 'sadness']
Sorry but your kids are crazy because that's how they were raised, and we get to deal with that every day, thanks!
1) anger 0.5995
2) optimism 0.213
3) sadness 0.1302
4) joy 0.0572
[0.96682936 0.0066428  0.01227691 0.01425079] ['anger', 'joy', 'optimism', 'sadness']
Not a drill. I have a tool kit in my classroom. I passed out hammers and wrenches and screw drivers. I was a drill sergeant. YOU flip the table up against the door. YOU here pass out scissors. YOU grab that chair. 
The kids are freshmen in high school. They were ready to fight. We were ready to fight for an hour and a half while we waited in the dark. I cussed a lot. We had a couple giggles. I was absolutely terrified I would

In [76]:
df.iloc[25]

post_id                                                    26
body_text    [www.mrsbrown.art/drive](www.mrsbrown.art/drive)
anger                                                0.178401
joy                                                  0.460689
optimism                                             0.216743
sadness                                              0.144167
Name: 25, dtype: object

In [74]:
len(df)

60